# Building Makemore MLP Exercise

## Imports

In [ ]:
from tqdm import tqdm
import numpy
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plot
import random
import math

In [ ]:
g = torch.Generator().manual_seed(42)

## Setup

In [ ]:
words = open('../data/names.txt', 'r').read().splitlines()
words[:8]

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [ ]:
len(words)

32033

In [ ]:
def generate_training_set(words, block_size, print_disabled=False):
    
    chars = sorted(list(set(''.join(words))))
    stoi = {s: i+1 for i, s in enumerate(chars)}
    stoi['.'] = 0
    itos = {i:s for s, i in stoi.items()}
    
    X, Y = [], []
    
    for w in words:
        if print_disabled: print(w)
        
        context = [0] * block_size
        for ch in w + '.':
            ix = stoi[ch]
            X.append(context)
            Y.append(ix)
            if print_disabled: print(''.join(itos[i] for i in context), '--->', itos[ix])
            context = context[1:] + [ix] # crop and append
            
    X = torch.tensor(X)
    Y = torch.tensor(Y)
    return X, Y

In [ ]:
X, Y = generate_training_set(words, 3)

In [ ]:
X.shape, Y.shape

(torch.Size([228146, 3]), torch.Size([228146]))

In [ ]:
def generate_train_valid_test_split(words, block_size=3):
    random.seed(42)
    random.shuffle(words)
    n1 = int(0.8*len(words))
    n2 = int(0.9*len(words))

    Xtr, Ytr = generate_training_set(words[:n1], block_size)
    Xdev, Ydev = generate_training_set(words[n1:n2], block_size)
    Xte, Yte = generate_training_set(words[n2:], block_size)
    
    return Xtr, Ytr, Xdev, Ydev, Xte, Yte

In [ ]:
Xtr, Ytr, Xdev, Ydev, Xte, Yte = generate_train_valid_test_split(words, block_size=3)

In [ ]:
Xtr.shape, Ytr.shape

(torch.Size([182625, 3]), torch.Size([182625]))

In [ ]:
Xdev.shape, Ydev.shape

(torch.Size([22655, 3]), torch.Size([22655]))

In [ ]:
Xte.shape, Yte.shape

(torch.Size([22866, 3]), torch.Size([22866]))

## E01

Tune the hyperparameters of the training to beat the validation loss of 2.2

   - no of neurons in the hidden layer
    
   - embedding size
    
   - no of characters
    
   - epochs
    
   - learning rate; change/decay it over the epochs
    
   - batch size

In [ ]:
def evaluate_loss(parameters, X, Y, block_size=3, embedding_size=10):
    C, W1, b1, W2, b2 = parameters
    emb = C[X]
    h = torch.tanh(emb.view(-1, block_size * embedding_size) @ W1 + b1)
    logits = h @ W2 + b2
    loss = F.cross_entropy(logits, Y)
    return loss

In [ ]:
def _regularization_loss(parameters, lambdas):
    C = parameters[0]
    W1 = parameters[1]
    W2 = parameters[3]
    
    return lambdas[0]*(C**2).mean() + lambdas[1]*(W1**2).mean() + lambdas[2]*(W2**2).mean()

In [ ]:
def train(X, 
          Y, 
          epochs, 
          block_size=3, 
          embedding_size=10, 
          hidden_neuron=300, 
          bs=32, 
          lr=0.1, 
          parameters=[], 
          lambdas = [0, 0, 0],
          enable_print=True,
          print_at_every_nth_epoch=10000
         ):
    
    if not parameters:
        C = torch.randn((27, embedding_size), generator=g)
        W1 = torch.randn((block_size * embedding_size, hidden_neuron), generator=g)
        b1 = torch.randn(hidden_neuron, generator=g)
        W2 = torch.randn((hidden_neuron, 27), generator=g)
        b2 = torch.randn(27, generator=g)
        parameters = [C, W1, b1, W2, b2]

    
    for p in parameters: p.requires_grad = True 
        
    for epoch in tqdm(range(epochs)):
            
        ix = torch.randint(0, X.shape[0], (bs, ))

        loss = evaluate_loss(parameters, X[ix], Y[ix], block_size, embedding_size)
        regularization_loss = _regularization_loss(parameters, lambdas)
        loss += regularization_loss

        for p in parameters:
            p.grad= None
        loss.backward()


        for p in parameters:
            p.data += - lr * p.grad

        if enable_print and epoch % print_at_every_nth_epoch == 0: print(epoch, loss.item())
    
    return parameters, loss.item()

### 1st try

In [ ]:
parameters, loss = train(Xtr, Ytr, 100_000, block_size=3, embedding_size=50, hidden_neuron=100, bs=16384, lr=0.1, enable_print=True, print_at_every_nth_epoch=10_000)

  0%|                                                                                                                                                                                                                    | 4/100000 [00:00<1:36:08, 17.34it/s]

0 17.771562576293945


 10%|████████████████████▊                                                                                                                                                                                           | 10003/100000 [08:59<1:19:38, 18.83it/s]

10000 2.3100812435150146


 20%|█████████████████████████████████████████▌                                                                                                                                                                      | 20003/100000 [17:47<1:10:41, 18.86it/s]

...

 80%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍                                         | 80005/100000 [1:24:15<16:17, 20.45it/s]

80000 2.1180062294006348


 90%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▏                    | 90003/100000 [1:32:20<08:05, 20.60it/s]

90000 2.1188645362854004


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100000/100000 [1:40:25<00:00, 16.60it/s]


In [ ]:
loss, evaluate_loss(parameters, Xdev, Ydev, block_size=3, embedding_size=50)

(2.101083755493164, tensor(2.1680, grad_fn=<NllLossBackward0>))

### 2nd try

In [ ]:
parameters, loss = train(Xtr, Ytr, 300_000, block_size=3, embedding_size=50, hidden_neuron=100, bs=16384, lr=0.01, parameters=parameters, enable_print=False)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 300000/300000 [4:07:07<00:00, 20.23it/s]


In [ ]:
loss, evaluate_loss(parameters, Xdev, Ydev, block_size=3, embedding_size=50)

(2.1126763820648193, tensor(2.1603, grad_fn=<NllLossBackward0>))

### 3rd try

In [ ]:
parameters, loss = train(Xtr, Ytr, 10_000, block_size=3, embedding_size=50, hidden_neuron=100, bs=16384, lr=1, parameters=parameters, enable_print=True, print_at_every_nth_epoch=1000)

  0%|                                                                                                                                                                                                                       | 4/10000 [00:00<08:24, 19.80it/s]

0 2.1231608390808105


 10%|█████████████████████▎                                                                                                                                                                                              | 1003/10000 [00:49<07:26, 20.17it/s]

1000 2.1130003929138184


 20%|██████████████████████████████████████████▍                                                                                                                                                                         | 2003/10000 [01:39<06:38, 20.07it/s]

...

 80%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋                                          | 8003/10000 [06:35<01:37, 20.42it/s]

8000 2.13120698928833


 90%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉                     | 9004/10000 [07:24<00:48, 20.40it/s]

9000 2.1335060596466064


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [08:13<00:00, 20.24it/s]


In [ ]:
loss, evaluate_loss(parameters, Xdev, Ydev, block_size=3, embedding_size=50)

(2.190765142440796, tensor(2.1794, grad_fn=<NllLossBackward0>))

### 4th try

In [ ]:
parameters, loss = train(Xtr, Ytr, 10_000, block_size=3, embedding_size=50, hidden_neuron=100, bs=16384, lr=0.1, parameters=parameters, enable_print=True, print_at_every_nth_epoch=1000)

  0%|                                                                                                                                                                                                                       | 5/10000 [00:00<08:24, 19.82it/s]

0 2.141832113265991


 10%|█████████████████████▎                                                                                                                                                                                              | 1005/10000 [00:49<07:20, 20.43it/s]

1000 2.091341495513916


 20%|██████████████████████████████████████████▌                                                                                                                                                                         | 2005/10000 [01:38<06:34, 20.29it/s]

...

 80%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋                                          | 8003/10000 [06:35<01:39, 20.12it/s]

8000 2.0842249393463135


 90%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉                     | 9004/10000 [07:24<00:47, 21.10it/s]

9000 2.0907206535339355


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [08:13<00:00, 20.25it/s]


In [ ]:
loss, evaluate_loss(parameters, Xdev, Ydev, block_size=3, embedding_size=50)

(2.090895414352417, tensor(2.1472, grad_fn=<NllLossBackward0>))

### 5th try

In [ ]:
parameters, loss = train(Xtr, Ytr, 100_000, block_size=3, embedding_size=50, hidden_neuron=100, bs=16384, lr=0.01, parameters=parameters, enable_print=True, print_at_every_nth_epoch=10_000)

  0%|                                                                                                                                                                                                                    | 5/100000 [00:00<1:23:39, 19.92it/s]

0 2.0824689865112305


 10%|████████████████████▊                                                                                                                                                                                           | 10004/100000 [08:12<1:14:28, 20.14it/s]

10000 2.0864546298980713


 20%|█████████████████████████████████████████▌                                                                                                                                                                      | 20005/100000 [16:24<1:05:10, 20.45it/s]

...

 80%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍                                         | 80004/100000 [1:05:29<16:18, 20.43it/s]

80000 2.095008611679077


 90%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▏                    | 90003/100000 [1:13:40<08:24, 19.82it/s]

90000 2.0829107761383057


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100000/100000 [1:21:51<00:00, 20.36it/s]


In [ ]:
loss, evaluate_loss(parameters, Xdev, Ydev, block_size=3, embedding_size=50)

(2.0964395999908447, tensor(2.1466, grad_fn=<NllLossBackward0>))

### Test Loss

In [ ]:
loss, evaluate_loss(parameters, Xte, Yte, block_size=3, embedding_size=50)

(2.0964395999908447, tensor(2.1446, grad_fn=<NllLossBackward0>))

## E02
- Weight Initialization

(1) What is the loss you'd get if the predicted probabilities at initialization were perfectly uniform? What loss do we achieve? 

(2) Can you tune the initialization to get a starting loss that is much more similar to (1)?

Answer to (1)

If the predicted probabilities were uniform then the probabilities would have been `1/27` of each character prediction

And we would have take the log of the probability which would have been

In [ ]:
torch.tensor(1/27).log()

tensor(-3.2958)

to the get the loss it would have been

In [ ]:
- torch.tensor(1/27).log()

tensor(3.2958)

No we sum up the losses and divide by the count, `(n * (3.2958))/n`
which is equal to `3.2958`

Lets see the initial loss when we train the model with current initialization

In [ ]:
parameters, loss = train(Xtr, Ytr, 10, block_size=3, embedding_size=50, hidden_neuron=100, bs=16384, lr=0.1, enable_print=True, print_at_every_nth_epoch=1)

 40%|███████████████████████████████████████████████████████████████████████████████████████▏                                                                                                                                  | 4/10 [00:00<00:00, 19.51it/s]

0 18.27653694152832
1 17.431493759155273
2 16.35456085205078
3 16.05698585510254


 90%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▏                     | 9/10 [00:00<00:00, 19.94it/s]

4 15.747321128845215
5 15.394339561462402
6 15.205368995666504
7 14.835010528564453
8 14.528204917907715


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 19.88it/s]

9 14.28638744354248


The initial loss is `18.98` which is high comparative to `3.2958`

Lets see the probabilities of the output

In [ ]:
parameters, loss = train(Xtr, Ytr, 1, block_size=3, embedding_size=50, hidden_neuron=100, bs=16384, lr=0.1, enable_print=True, print_at_every_nth_epoch=1)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 19.37it/s]

0 18.204519271850586


In [ ]:
def compute_probs(parameters, X, block_size=3, embedding_size=50):
    C, W1, b1, W2, b2 = parameters
    emb = C[X]
    h = torch.tanh(emb.view(-1, block_size * embedding_size) @ W1 + b1)
    logits = h @ W2 + b2
    return F.softmax(logits, dim=1)

In [ ]:
compute_probs(parameters, Xtr)

tensor([[2.9970e-06, 2.3740e-08, 2.1316e-10,  ..., 1.2648e-13, 8.5370e-04,
         8.7376e-08],
        [1.7422e-05, 1.1364e-09, 1.3196e-09,  ..., 3.6301e-13, 3.8613e-06,
         2.4013e-07],
        [5.8833e-05, 5.7244e-06, 1.0801e-02,  ..., 9.2642e-07, 2.9683e-06,
         2.4511e-06],
        ...,
        [5.7658e-11, 1.4429e-09, 9.7899e-11,  ..., 1.0416e-11, 8.2188e-09,
         2.7279e-10],
        [7.0990e-01, 8.7623e-12, 1.6534e-07,  ..., 3.1374e-09, 3.6852e-06,
         1.1986e-04],
        [9.9999e-01, 1.0279e-07, 6.2436e-11,  ..., 1.4053e-10, 6.7408e-14,
         1.2024e-09]], grad_fn=<SoftmaxBackward0>)

Lets view a single row of probabilities

In [ ]:
compute_probs(parameters, Xtr)[0]

tensor([2.9970e-06, 2.3740e-08, 2.1316e-10, 1.9171e-08, 3.7981e-04, 2.2313e-02,
        1.3911e-17, 1.0186e-09, 9.7561e-10, 5.6293e-12, 8.8295e-09, 3.4877e-09,
        1.2439e-08, 7.9825e-14, 7.3846e-04, 1.0648e-11, 5.4885e-08, 3.0407e-13,
        2.0024e-02, 9.5325e-01, 1.7357e-03, 2.2441e-08, 6.8103e-04, 2.4685e-05,
        1.2648e-13, 8.5370e-04, 8.7376e-08], grad_fn=<SelectBackward0>)

to get a uniform probability, I think we need to have all logits as equal so that we can get probability of each as `1/27`

### Try 1

lets try uniform wieght initialization

In [ ]:
def train_v2(X, 
          Y, 
          epochs, 
          block_size=3, 
          embedding_size=10, 
          hidden_neuron=300, 
          bs=32, 
          lr=0.1, 
          parameters=[], 
          lambdas = [0, 0, 0],
          enable_print=True,
          print_at_every_nth_epoch=10000
         ):
    
    if not parameters:
        C = torch.rand((27, embedding_size), generator=g)
        W1 = torch.rand((block_size * embedding_size, hidden_neuron), generator=g)
        b1 = torch.rand(hidden_neuron, generator=g)
        W2 = torch.rand((hidden_neuron, 27), generator=g)  
        b2 = torch.rand(27, generator=g)
        parameters = [C, W1, b1, W2, b2]

    
    for p in parameters: p.requires_grad = True 
        
    for epoch in tqdm(range(epochs)):
            
        ix = torch.randint(0, X.shape[0], (bs, ))

        loss = evaluate_loss(parameters, X[ix], Y[ix], block_size, embedding_size)
        regularization_loss = _regularization_loss(parameters, lambdas)
        loss += regularization_loss

        for p in parameters:
            p.grad= None
        loss.backward()


        for p in parameters:
            p.data += - lr * p.grad

        if enable_print and epoch % print_at_every_nth_epoch == 0: print(epoch, loss.item())
    
    return parameters, loss.item()

In [ ]:
parameters, loss = train_v2(Xtr, Ytr, 1, block_size=3, embedding_size=50, hidden_neuron=100, bs=16384, lr=0.1, enable_print=True, print_at_every_nth_epoch=1)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 20.50it/s]

0 7.901321887969971


With uniform weight initialization the intial loss (`6.422`) obtained is less than of normal weight initialization (`17.7`)

### Try 2

Lets initialize the last layers of weights and biases as zero.

In [ ]:
def train_v3(X, 
          Y, 
          epochs, 
          block_size=3, 
          embedding_size=10, 
          hidden_neuron=300, 
          bs=32, 
          lr=0.1, 
          parameters=[], 
          lambdas = [0, 0, 0],
          enable_print=True,
          print_at_every_nth_epoch=10000
         ):
    
    if not parameters:
        C = torch.rand((27, embedding_size), generator=g)
        W1 = torch.rand((block_size * embedding_size, hidden_neuron), generator=g)
        b1 = torch.rand(hidden_neuron)
        W2 = torch.zeros((hidden_neuron, 27))
        b2 = torch.zeros(27)
        parameters = [C, W1, b1, W2, b2]

    
    for p in parameters: p.requires_grad = True 
        
    for epoch in tqdm(range(epochs)):
            
        ix = torch.randint(0, X.shape[0], (bs, ))

        loss = evaluate_loss(parameters, X[ix], Y[ix], block_size, embedding_size)
        regularization_loss = _regularization_loss(parameters, lambdas)
        loss += regularization_loss

        for p in parameters:
            p.grad= None
        loss.backward()


        for p in parameters:
            p.data += - lr * p.grad

        if enable_print and epoch % print_at_every_nth_epoch == 0: print(epoch, loss.item())
    
    return parameters, loss.item()

In [ ]:
parameters, loss = train_v3(Xtr, Ytr, 1, block_size=3, embedding_size=50, hidden_neuron=100, bs=16384, lr=0.1, enable_print=True, print_at_every_nth_epoch=1)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 21.42it/s]

0 3.295837163925171


The initial loss is now `3.2958` (which we wanted). 

Lets see how well it trains now

In [ ]:
parameters, loss = train_v3(Xtr, Ytr, 30_000, block_size=3, embedding_size=50, hidden_neuron=100, bs=16384, lr=0.1, enable_print=True, print_at_every_nth_epoch=10_000)

  0%|                                                                                                                                                                                                                               | 0/30000 [00:00<?, ?it/s]

0 3.295837163925171


 33%|██████████████████████████████████████████████████████████████████████▎                                                                                                                                            | 10005/30000 [07:59<15:49, 21.07it/s]

10000 2.8236281871795654


 67%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋                                                                      | 20004/30000 [15:58<08:02, 20.72it/s]

20000 2.8253743648529053


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 30000/30000 [23:57<00:00, 20.86it/s]


In [ ]:
loss

2.8211419582366943

### Try 3

As we can see the losses are not decreasing faster, lets not initialize weight to zero but close to zero and see ...

In [ ]:
def train_v4(X, 
          Y, 
          epochs, 
          block_size=3, 
          embedding_size=10, 
          hidden_neuron=300, 
          bs=32, 
          lr=0.1, 
          parameters=[], 
          lambdas = [0, 0, 0],
          enable_print=True,
          print_at_every_nth_epoch=10000
         ):
    
    if not parameters:
        C = torch.rand((27, embedding_size), generator=g)
        W1 = torch.rand((block_size * embedding_size, hidden_neuron), generator=g)
        b1 = torch.rand(hidden_neuron)
        W2 = torch.rand((hidden_neuron, 27)) * 0.01 # close to zero
        b2 = torch.zeros(27)
        parameters = [C, W1, b1, W2, b2]

    
    for p in parameters: p.requires_grad = True 
        
    for epoch in tqdm(range(epochs)):
            
        ix = torch.randint(0, X.shape[0], (bs, ))

        loss = evaluate_loss(parameters, X[ix], Y[ix], block_size, embedding_size)
        regularization_loss = _regularization_loss(parameters, lambdas)
        loss += regularization_loss

        for p in parameters:
            p.grad= None
        loss.backward()


        for p in parameters:
            p.data += - lr * p.grad

        if enable_print and epoch % print_at_every_nth_epoch == 0: print(epoch, loss.item())
    
    return parameters, loss.item()

In [ ]:
parameters, loss = train_v4(Xtr, Ytr, 30_000, block_size=3, embedding_size=50, hidden_neuron=100, bs=16384, lr=0.1, enable_print=True, print_at_every_nth_epoch=10_000)

  0%|                                                                                                                                                                                                                       | 3/30000 [00:00<23:51, 20.95it/s]

0 3.29825496673584


 33%|██████████████████████████████████████████████████████████████████████▎                                                                                                                                            | 10005/30000 [08:00<15:48, 21.08it/s]

10000 2.8132688999176025


 67%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋                                                                      | 20003/30000 [16:00<08:02, 20.73it/s]

20000 2.826235294342041


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 30000/30000 [24:00<00:00, 20.82it/s]


### Try 4

Lets not try to uniformly initiate all the weights but only the last layers and the rest we can keep as normal initialized

In [ ]:
def train_v5(X, 
          Y, 
          epochs, 
          block_size=3, 
          embedding_size=10, 
          hidden_neuron=300, 
          bs=32, 
          lr=0.1, 
          parameters=[], 
          lambdas = [0, 0, 0],
          enable_print=True,
          print_at_every_nth_epoch=10000
         ):
    
    if not parameters:
        C = torch.randn((27, embedding_size), generator=g)
        W1 = torch.randn((block_size * embedding_size, hidden_neuron), generator=g)
        b1 = torch.randn(hidden_neuron)
        W2 = torch.rand((hidden_neuron, 27)) * 0.01 # close to zero
        b2 = torch.zeros(27)
        parameters = [C, W1, b1, W2, b2]

    
    for p in parameters: p.requires_grad = True 
        
    for epoch in tqdm(range(epochs)):
            
        ix = torch.randint(0, X.shape[0], (bs, ))

        loss = evaluate_loss(parameters, X[ix], Y[ix], block_size, embedding_size)
        regularization_loss = _regularization_loss(parameters, lambdas)
        loss += regularization_loss

        for p in parameters:
            p.grad= None
        loss.backward()


        for p in parameters:
            p.data += - lr * p.grad

        if enable_print and epoch % print_at_every_nth_epoch == 0: print(epoch, loss.item())
    
    return parameters, loss.item()

In [ ]:
parameters, loss = train_v5(Xtr, Ytr, 30_000, block_size=3, embedding_size=50, hidden_neuron=100, bs=16384, lr=0.1, enable_print=True, print_at_every_nth_epoch=10_000)

  0%|                                                                                                                                                                                                                       | 3/30000 [00:00<24:39, 20.28it/s]

0 3.2991583347320557


 33%|██████████████████████████████████████████████████████████████████████▎                                                                                                                                            | 10003/30000 [08:13<16:37, 20.04it/s]

10000 2.175701379776001


 67%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋                                                                      | 20005/30000 [16:28<08:11, 20.33it/s]

20000 2.1791296005249023


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 30000/30000 [24:42<00:00, 20.24it/s]


The losses are reducing now. Lets train for 100_000 and check

In [ ]:
parameters, loss = train_v5(Xtr, Ytr, 200_000, block_size=3, embedding_size=50, hidden_neuron=100, bs=16384, lr=0.1, enable_print=True, print_at_every_nth_epoch=10_000)

  0%|                                                                                                                                                                                                                    | 4/200000 [00:00<3:01:12, 18.39it/s]

0 3.292088270187378


  5%|██████████▍                                                                                                                                                                                                     | 10004/200000 [08:33<2:43:37, 19.35it/s]

10000 2.184924364089966


 10%|████████████████████▊                                                                                                                                                                                           | 20002/200000 [16:56<2:28:24, 20.21it/s]

...

 90%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎                    | 180003/200000 [2:31:25<16:36, 20.06it/s]

180000 2.0731143951416016


 95%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋          | 190005/200000 [2:39:51<08:15, 20.16it/s]

190000 2.0831706523895264


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 200000/200000 [2:48:07<00:00, 19.83it/s]


In [ ]:
loss

2.0791072845458984

The losses are getting reduced faster!

In [ ]:
evaluate_loss(parameters, Xdev, Ydev, block_size=3, embedding_size=50)

tensor(2.1343, grad_fn=<NllLossBackward0>)

## E03

Read the Bengio et al 2003 paper, implement and try any idea from the paper. Did it work?

In the paper there is a mention of direct connection from the word features to output. 

Lets implement the direct connection from embedding to output and see the results

### Direct connection from embedding to output

In [ ]:
C = torch.randn((27, 50), generator=g)

In [ ]:
C[X].shape; C[X].view(-1, 150).shape

torch.Size([228146, 150])

In [ ]:
def evaluate_loss_dir_conn(parameters, X, Y, block_size=3, embedding_size=10):
    C, W1, b1, W2, W3, b2 = parameters
    emb = C[X]
    h = torch.tanh(emb.view(-1, block_size * embedding_size) @ W1 + b1)
    logits = h @ W2 + b2 + C[X].view(-1, block_size * embedding_size) @ W3
    loss = F.cross_entropy(logits, Y)
    return loss

In [ ]:
def train_dir_conn(X, 
          Y, 
          epochs, 
          block_size=3, 
          embedding_size=10, 
          hidden_neuron=300, 
          bs=32, 
          lr=0.1, 
          parameters=[], 
          lambdas = [0, 0, 0],
          enable_print=True,
          print_at_every_nth_epoch=10000
         ):
    
    if not parameters:
        C = torch.randn((27, embedding_size), generator=g)
        W1 = torch.randn((block_size * embedding_size, hidden_neuron), generator=g)
        b1 = torch.randn(hidden_neuron)
        W2 = torch.rand((hidden_neuron, 27)) * 0.01 # close to zero
        W3 = torch.rand((block_size * embedding_size, 27)) * 0.01 # close to zero
        b2 = torch.zeros(27)
        parameters = [C, W1, b1, W2, W3, b2]

    
    for p in parameters: p.requires_grad = True 
        
    for epoch in tqdm(range(epochs)):
            
        ix = torch.randint(0, X.shape[0], (bs, ))

        loss = evaluate_loss_dir_conn(parameters, X[ix], Y[ix], block_size, embedding_size)
        regularization_loss = _regularization_loss(parameters, lambdas)
        loss += regularization_loss

        for p in parameters:
            p.grad= None
        loss.backward()


        for p in parameters:
            p.data += - lr * p.grad

        if enable_print and epoch % print_at_every_nth_epoch == 0: print(epoch, loss.item())
    
    return parameters, loss.item()

In [ ]:
parameters, loss = train_dir_conn(Xtr, Ytr, 100_000, block_size=3, embedding_size=50, hidden_neuron=100, bs=16384, lr=0.1, enable_print=True, print_at_every_nth_epoch=10_000)

  0%|                                                                                                                                                                                                                    | 2/100000 [00:00<2:09:58, 12.82it/s]

0 3.29349684715271


 10%|████████████████████▊                                                                                                                                                                                           | 10002/100000 [11:49<1:43:19, 14.52it/s]

10000 2.151575803756714


 20%|█████████████████████████████████████████▌                                                                                                                                                                      | 20002/100000 [23:20<1:35:02, 14.03it/s]

...

 80%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍                                         | 80002/100000 [1:33:29<22:48, 14.61it/s]

80000 2.0761640071868896


 90%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▏                    | 90002/100000 [1:44:59<11:26, 14.57it/s]

90000 2.0695760250091553


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100000/100000 [1:56:30<00:00, 14.31it/s]


In [ ]:
loss

2.0880658626556396

In [ ]:
evaluate_loss_dir_conn(parameters, Xdev, Ydev, block_size=3, embedding_size=50)

tensor(2.1274, grad_fn=<NllLossBackward0>)

In [ ]:
evaluate_loss_dir_conn(parameters, Xte, Yte, block_size=3, embedding_size=50)

tensor(2.1239, grad_fn=<NllLossBackward0>)

The loss decreased by lot with this direct connection and the above method of weight initialization